In [1]:
import numpy as np
import matplotlib.pyplot as plt
import time
import os
import h5py

#in-house functions for directed actflow analysis and visualization
import pseudoEmpiricalData
import featureSelectRegression
import plot_activations as plt_act

#to call the PC algorithm Java package
#(be sure you have the tetrad_causal-cmd-1.1.3 directory)
import PCalg_wrapper 

import CombinedFC.CombinedFCToolBox as cfc #original repo: https://github.com/ColeLab/CombinedFC
import ActflowToolbox as actflow #original repo: https://github.com/ColeLab/ActflowToolbox

### download HCP empirical rs-bold data and task activations

In [2]:
data_dir = f'{os.getcwd()}/hcp_data' 

# hcp subjects by ID: 176 subjects
# Datasets contain for each subject: 
# - one session resting-state bold [360 regions x 1195 datapoints]
# - 24 task conditions activations [360 regions x 24 conditions]
# 360 regions are from the Glasser et al. (2016) parcellation

hcp_cohort = ['100206','108020','117930','126325','133928','143224','153934','164636','174437',
            '183034','194443','204521','212823','268749','322224','385450','463040','529953',
            '587664','656253','731140','814548','877269','978578','100408','108222','118124',
            '126426','134021','144832','154229','164939','175338','185139','194645','204622',
            '213017','268850','329844','389357','467351','530635','588565','657659','737960',
            '816653','878877','987074','101006','110007','118225','127933','134324','146331',
            '154532','165638','175742','185341','195445','205119','213421','274542','341834',
            '393247','479762','545345','597869','664757','742549','820745','887373','989987',
            '102311','111009','118831','128632','135528','146432','154936','167036','176441',
            '186141','196144','205725','213522','285345','342129','394956','480141','552241',
            '598568','671855','744553','826454','896879','990366','102513','112516','118932',
            '129028','135629','146533','156031','167440','176845','187850','196346','205826',
            '214423','285446','348545','395756','481042','553344','599671','675661','749058',
            '832651','899885','991267','102614','112920','119126','129129','135932','147636',
            '157336','168745','177645','188145','198350','208226','214726','286347','349244',
            '406432','486759','555651','604537','679568','749361','835657','901442','992774',
            '103111','113316','120212','130013','136227','148133','157437','169545','178748',
            '188549','198451','208327','217429','290136','352738','414229','497865','559457',
            '615744','679770','753150','837560','907656','993675','103414','113619','120414',
            '130114','136833','150726','157942','171330']

num_subjects = len(hcp_cohort)
num_regions = 360
num_tasks = 24  #24 task conditions
num_datapoints = 1195
#initialize an array to save subject level data
rsbold_data = np.empty((num_regions,num_datapoints,num_subjects))
activation_data = np.empty((num_regions,num_tasks,num_subjects))

for s in range(num_subjects):
    # read the h5 files from the hcp_data directory
    hf = h5py.File(f'{data_dir}/hcp_{hcp_cohort[s]}_rsBold_taskActivations.h5', 'r')
    # save the resting-state bold data
    rsbold_data[:,:,s] = hf.get('rest_bold')
    # save the actual activations for the task conditions
    activation_data[:,:,s] = hf.get('taskByConditions_activations')

### Functional connectivity (FC) analysis + Activity flow predictions

### correlation

In [3]:
%%time
# correlation analysis of the resting-state data
#save FC matrices per subject
m_corr = np.empty((num_regions,num_regions,num_subjects))
# save activity flow predictions per region, per task, per subject
pred_act_corr = np.empty((num_regions,num_tasks,num_subjects))
# save running times to evaluate computational efficiency
time_corr = np.empty((num_subjects,))

for subj in range(num_subjects):
    start = time.time()
    # FC computation
    m_corr[:,:,subj] = cfc.correlationSig(rsbold_data[:,:,subj].T, alpha = 0.01)
    
    # activity flow calculation using activity flow toolbox
    for j in range(num_tasks):
        pred_act_corr[:,j,subj] = actflow.actflowcalc(activation_data[:,j,subj], m_corr[:,:,subj])
    
    # save running time
    time_corr[subj] = time.time() - start

print('- correlation FC matrices and activity flow predictions')
print(f'- for {num_regions} regions, {num_tasks} task conditions and {num_subjects} subjects')

- correlation FC matrices and activity flow predictions
- for 360 regions, 24 task conditions and 176 subjects
CPU times: user 1min 58s, sys: 574 ms, total: 1min 59s
Wall time: 1min 27s


### multiple regression

In [ ]:
%%time
#multiple regression analysis of the resting-state data
m_mreg = np.empty((num_regions,num_regions,num_subjects))
pred_act_mreg = np.empty((num_regions,num_tasks,num_subjects))
time_mreg = np.empty((num_subjects,))

for subj in range(num_subjects):
    start = time.time()
    # FC computation
    m_mreg[:,:,subj] = cfc.multipleRegressionSig(rsbold_data[:,:,subj].T, alpha = 0.01, sigTest = True)
    
    # activity flow calculation using activity flow toolbox
    for j in range(num_tasks):
        pred_act_mreg[:,j,subj] = actflow.actflowcalc(activation_data[:,j,subj], m_mreg[:,:,subj])
    
    #save running time
    time_mreg[subj] = time.time() - start

print('- multiple regression FC matrices and activity flow predictions')
print(f'- for {num_regions} regions, {num_tasks} task conditions and {num_subjects} subjects')

In [ ]:
%%time
#combinedFC with feature selection analysis of resting-state data X_rest
m_cfc =  np.empty((num_regions,num_regions,num_subjects))
pred_act_cfc = np.empty((num_regions,num_tasks,num_subjects))
time_cfc = np.empty((num_subjects,))

for subj in range(num_subjects):
    start = time.time()
    # FC computation
    data = rsbold_data[:,:,subj].T
    M_aux = cfc.combinedFC(data,
                         methodCondAsso = 'partialCorrelation',
                         alphaCondAsso = 0.01,
                         methodAsso = 'correlation',
                         alphaAsso = 0.01)
    
    m_cfc[:,:,subj] = featureSelectRegression.featureSelectRegression(M_aux, data, typ='symmetric')
    
    # activity flow calculation using activity flow toolbox    
    for j in range(num_tasks):
        pred_act_cfc[:,j,subj] = actflow.actflowcalc(activation_data[:,j,subj], m_cfc[:,:,subj])
    
    #save running time
    time_cfc[subj] = time.time() - start
    
print('- combinedFC FC matrices and activity flow predictions')
print(f'- for {num_regions} regions, {num_tasks} task conditions and {num_subjects} subjects')

In [ ]:
%%time
#PC algorithm FC analysis of resting-state data X_rest
m_PC =  np.empty((num_regions,num_regions,num_subjects))
pred_act_PC = np.empty((num_regions,num_tasks,num_subjects))
time_PC = np.empty((num_subjects,))
#PC-adjacencies
m_PCadj =  np.empty((num_regions,num_regions,num_subjects))
pred_act_PCadj = np.empty((num_regions,num_tasks,num_subjects))
time_PCadj = np.empty((num_subjects,))

current_dir = os.getcwd()

# large continuous runs may "stuck" Tetrad (not clear why)
# a cheap solution is to run the repetitions in batches

batches = np.arange(0,num_subjects,5)
subj = 0
for bat in range(len(batches)):
    for a in range(batches[bat],batches[bat]+5):
        startPC = time.time()
        # FC computation
        # temporary save data for PC. The algorithm requires a txt file as input
        data = rsbold_data[:,:,subj].T
        # reuse the same name to avoid ending up with too many txt files
        inputData = f'{current_dir}/temp/data_temp.txt'
        np.savetxt(inputData, data, fmt='%0.6f', delimiter=',')
        # define output directory, output file name and Tetrad software path
        outDir = f'{current_dir}/temp'
        outName = 'PCgraph_temp'
        tetrad_path = f'{current_dir}/tetrad_causal-cmd-1.1.3/causal-cmd-1.1.3-jar-noMeekRules234.jar'
        # call PC
        PCalg_wrapper.PCalg_run(inputData, outDir, outName, tetrad_path, alpha = 0.01)
        # transform the PC output graph into a numpy array
        PC_graph = PCalg_wrapper.tetrad2matrix(f'{outDir}/{outName}.txt')
        time_search_PC = time.time() - startPC
        print(f'graph for subject {subj}')

        # Use the set of parents as regressors y = bPa(X) + e, to get a weighted directed matrix
        # refer as PC or PC algorithm in the paper
        start = time.time()
        m_PC[:,:,subj] = featureSelectRegression.featureSelectRegression(PC_graph, data, typ='parents')
        # activity flow calculation using activity flow toolbox
        for j in range(num_tasks):
            pred_act_PC[:,j,subj] = actflow.actflowcalc(activation_data[:,j,subj], m_PC[:,:,subj])
        # running time is the PC search time + computation of weights time + actflowcalc    
        time_PC[subj] = (time.time() - start) + time_search_PC

        # Use the adjacencies of the PC results: parents and children: y = bPa(X) + cCh(x),
        # refer as PCadj or PC adjacencies, in the paper
        start = time.time()
        m_PCadj[:,:,subj] = featureSelectRegression.featureSelectRegression(PC_graph, data, typ='adjacencies')
        
        # activity flow calculation using activity flow toolbox
        for j in range(num_tasks): 
            pred_act_PCadj[:,j,subj] = actflow.actflowcalc(activation_data[:,j,subj], m_PCadj[:,:,subj])
        time_PCadj[subj] = (time.time() - start) + time_search_PC
    
        subj = subj + 1
        
# remove the tetrad-causal log file at the end of all the runs
# comment this if want to keep it
os.remove(f'{current_dir}/causal-cmd.log')

print('- PC and PC-adjacencies FC matrices and activity flow predictions')
print(f'- for {num_regions} regions, {num_tasks} task conditions and {num_subjects} subjects')

### compute prediction accuracy of the actflow models parameterized with the different FC methods

In [ ]:
# Output measures: Pearson correlation / R^2 coefficient of determination // MAE mean absolute error
# dict_keys(['corr_vals', 'R2_vals', 'mae_vals'])
# For each of the accuracy metrics the output is a 2D list: task conditions x number of subjects

# compute prediction accuracy of an activity flow model
# Input is a 3D activation array: region x task condition x subject
# computes the accuracy individually for each task condition across regions, per subject
corr_acc = actflow.model_compare_predicted_to_actual(
                                          target_actvect = activation_data,
                                          pred_actvect = pred_act_corr,
                                          comparison_type = 'nodewise_compthenavg'
                                          )

mreg_acc = actflow.model_compare_predicted_to_actual(
                                          target_actvect = activation_data,
                                          pred_actvect = pred_act_mreg,
                                          comparison_type = 'nodewise_compthenavg'
                                          )

cfc_acc = actflow.model_compare_predicted_to_actual(
                                          target_actvect = activation_data,
                                          pred_actvect = pred_act_cfc,
                                          comparison_type = 'nodewise_compthenavg'
                                          )

PCadj_acc = actflow.model_compare_predicted_to_actual(
                                          target_actvect = activation_data,
                                          pred_actvect = pred_act_PCadj,
                                          comparison_type = 'nodewise_compthenavg'
                                          )

PC_acc = actflow.model_compare_predicted_to_actual(
                                          target_actvect = activation_data,
                                          pred_actvect = pred_act_PC,
                                          comparison_type = 'nodewise_compthenavg'
                                          )

### plot actflow models prediction accuracies

In [ ]:
# choose Pearson r as accuracy metric: options: ['corr_vals', 'R2_vals', 'mae_vals']
# Accuracies are first averaged across task conditions
corr_acc_conditionAvg = np.mean(corr_acc['corr_vals'],axis=0)
mreg_acc_conditionAvg = np.mean(mreg_acc['corr_vals'],axis=0)
cfc_acc_conditionAvg = np.mean(cfc_acc['corr_vals'],axis=0)
PCadj_acc_conditionAvg = np.mean(PCadj_acc['corr_vals'],axis=0)
PC_acc_conditionAvg = np.mean(PC_acc['corr_vals'],axis=0)

# consolidate results for bar plotting
results_prediction = [corr_acc_conditionAvg,
                      mreg_acc_conditionAvg,
                      cfc_acc_conditionAvg,
                      PCadj_acc_conditionAvg,
                      PC_acc_conditionAvg]
# print median precision across subjects for each method
print(f'nodewise median prediction accuracy (r) across {num_tasks} task conditions and {num_subjects} subjects')
print(f' correlation   : {np.median(corr_acc_conditionAvg):.4f}')
print(f' mul.reg       : {np.median(mreg_acc_conditionAvg):.4f}')
print(f' combinedFC    : {np.median(cfc_acc_conditionAvg):.4f}')
print(f' PC-adjacencies: {np.median(PCadj_acc_conditionAvg):.4f}')
print(f' PC algorithm  : {np.median(PC_acc_conditionAvg):.4f}')
#plot
fig, ax = plt.subplots(figsize=(6.5,5)) 
bp = ax.boxplot(results_prediction,showfliers=False,patch_artist = True)
for box in bp['boxes']:
    # change fill color
    box.set( facecolor = 'white' )
for median in bp['medians']:
    median.set(color='green',linewidth=2)
plt.title('Actflow prediction accuracy', fontsize=20)
plt.ylabel('Pearson r',fontsize=20)
plt.yticks(fontsize=17)
plt.tick_params(axis='x',direction='inout',length=10)
plt.ylim(0,1)
plt.xticks(np.arange(1,6,1),('corr','mulReg','combFC','PCadj','PC'),fontsize=17,rotation=0)
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.set_facecolor('whitesmoke')
ax.grid(True,color='white',linewidth=2)
# uncomment to save plot as pdf
#plt.savefig('EmpiricalPredictionAccuracy.pdf',bbox_inches='tight')

### compare prediction accuracy with coefficient of determination R^2

In [ ]:
# choose coefficient of determination R^2 as accuracy metric: options: ['corr_vals', 'R2_vals', 'mae_vals']
# Accuracies are first averaged across task conditions
# print median precision across subjects for each method
# choose Pearson r as accuracy metric: options: ['corr_vals', 'R2_vals', 'mae_vals']
corr_acc_conditionAvg_R2 = np.mean(corr_acc['R2_vals'],axis=0)
mreg_acc_conditionAvg_R2 = np.mean(mreg_acc['R2_vals'],axis=0)
cfc_acc_conditionAvg_R2 = np.mean(cfc_acc['R2_vals'],axis=0)
PCadj_acc_conditionAvg_R2 = np.mean(PCadj_acc['R2_vals'],axis=0)
PC_acc_conditionAvg_R2 = np.mean(PC_acc['R2_vals'],axis=0)

# print median precision across subjects for each method
print(f'nodewise median prediction accuracy (R^2) across {num_tasks} task conditions and {num_subjects} subjects')
print(f' correlation   : {np.median(corr_acc_conditionAvg_R2):.4f}')
print(f' mul.reg       : {np.median(mreg_acc_conditionAvg_R2):.4f}')
print(f' combinedFC    : {np.median(cfc_acc_conditionAvg_R2):.4f}')
print(f' PC-adjacencies: {np.median(PCadj_acc_conditionAvg_R2):.4f}')
print(f' PC algorithm  : {np.median(PC_acc_conditionAvg_R2):.4f}')

### Plot number of predictors for activity flow prediction

In [ ]:
#number of possible predictors for to-be-predicted target region
#the more predictors (sources), the more complex, and less tractable is the predictive model
def numPredictors(inferred_model):
    num_subjects = inferred_model.shape[2]
    num_predictors = np.empty((num_subjects))
    for subj in range(num_subjects):
        #average of number of sources across all the regions in the inferred network
        num_predictors[subj] = np.mean(np.sum(inferred_model[:,:,subj]!=0,axis=1))   
    return num_predictors

In [ ]:
# define average number of predictors for each method, for the inferred FC network
corr_numPredictors = numPredictors(m_corr)
mreg_numPredictors = numPredictors(m_mreg)
cfc_numPredictors = numPredictors(m_cfc)
PCadj_numPredictors = numPredictors(m_PCadj)
PC_numPredictors = numPredictors(m_PC)
# consolidate results for bar plot
results_numPredictors = [corr_numPredictors,
                         mreg_numPredictors, 
                         cfc_numPredictors, 
                         PCadj_numPredictors, 
                         PC_numPredictors]
# print median number of predictors across subjects for each method
print(f'median number of predictors across {num_subjects} repetitions')
print(f' correlation   : {np.median(corr_numPredictors):.2f}')
print(f' mul.reg       : {np.median(mreg_numPredictors):.2f}')
print(f' combinedFC    : {np.median(cfc_numPredictors):.2f}')
print(f' PC-adjacencies: {np.median(PCadj_numPredictors):.2f}')
print(f' PC algorithm  : {np.median(PC_numPredictors):.2f}')

# plot
fig, ax = plt.subplots(figsize=(6.5,5)) 
bp = ax.boxplot(results_numPredictors,showfliers=False,patch_artist = True)
for box in bp['boxes']:
    # change fill color
    box.set( facecolor = 'white' )
for median in bp['medians']:
    median.set(color='green',linewidth=2)
plt.title('Number of predictors',fontsize=20)
plt.yticks(fontsize=17)
plt.tick_params(axis='x',direction='inout',length=10)
plt.yscale('log')
plt.ylabel('log scale', fontsize=20)
plt.xticks(np.arange(1,6,1),('corr','mulReg','combFC','PCadj','PC'),fontsize=17,rotation=0)
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.set_facecolor('whitesmoke')
ax.grid(True,color='white',linewidth=2)
# uncomment to save plot as pdf
#plt.savefig('EmpiricalNumberOfPredictors_log.pdf',bbox_inches='tight')

### plot running times

In [ ]:
#consolidate results for barplots
results_time = [time_corr, time_mreg, time_cfc,time_PCadj,time_PC]
#print median running times across repetitions for each method
print(f'median running time (sec) across {num_subjects} repetitions')
print(f' correlation   : {np.median(time_corr):.4f}')
print(f' mul.reg       : {np.median(time_mreg):.4f}')
print(f' combinedFC    : {np.median(time_cfc):.4f}')
print(f' PC-adjacencies: {np.median(time_PCadj):.4f}')
print(f' PC algorithm  : {np.median(time_PC):.4f}')
#plot
fig, ax = plt.subplots(figsize=(6.5,5)) 
bp = ax.boxplot(results_time,showfliers=False,patch_artist = True)
for box in bp['boxes']:
    # change fill color
    box.set( facecolor = 'white' )
for median in bp['medians']:
    median.set(color='green',linewidth=2)
plt.title('Running time',fontsize=20,y=1.02)
plt.yticks(fontsize=17)
plt.tick_params(axis='x',direction='inout',length=10)
plt.ylabel('seconds',fontsize=20)
plt.xticks(np.arange(1,6,1),('corr','mulReg','combFC','PCadj','PC'),fontsize=17,rotation=0)
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.set_facecolor('whitesmoke')
ax.grid(True,color='white',linewidth=2)
# uncomment to save plot as pdf
#plt.savefig('EmpiricalRunningTimes.pdf', bbox_inches='tight')

### plot FC connectivity matrices

In [ ]:
mat = m_corr
# plot average across subjects
mat = np.mean(mat,axis=2)
# threshold the plot for values +/- 0.005
m = abs(mat) > 0.005
mat = np.multiply(mat,m)
# use the combinedFC toolbox plotting functions
fig = cfc.plotConnectivityMatrix(mat,
                           methodTitle='Correlation FC',
                           functionalNetworks = True,
                           networkColorBar_x = True,
                           networkColorBar_y = True,
                           networkLabels = True,
                           )
# uncomment to save plot as pdf
#plt.savefig('EmpiricalCorrelationFC.pdf', bbox_inches='tight')

mat = m_mreg
mat = np.mean(mat,axis=2)
m = abs(mat) > 0.005
mat = np.multiply(mat,m)
fig = cfc.plotConnectivityMatrix(mat,
                           methodTitle='Multiple regression FC',
                           functionalNetworks = True,
                           networkColorBar_x = True,
                           networkColorBar_y = True,
                           networkLabels = True,
                           )
# uncomment to save plot as pdf
#plt.savefig('EmpiricalMultipleRegressionFC.pdf', bbox_inches='tight')


mat = m_cfc
mat = np.mean(mat,axis=2)
m = abs(mat) > 0.005
mat = np.multiply(mat,m)
fig = cfc.plotConnectivityMatrix(mat,
                           methodTitle='CombinedFC FC',
                           functionalNetworks = True,
                           networkColorBar_x = True,
                           networkColorBar_y = True,
                           networkLabels = True,
                           )
# uncomment to save plot as pdf
#plt.savefig('EmpiricalCombinedFCFC.pdf', bbox_inches='tight')

mat = m_PCadj
mat = np.mean(mat,axis=2)
m = abs(mat) > 0.005
mat = np.multiply(mat,m)
fig = cfc.plotConnectivityMatrix(mat,
                           methodTitle='PC-adjacencies FC',
                           functionalNetworks = True,
                           networkColorBar_x = True,
                           networkColorBar_y = True,
                           networkLabels = True,
                           )
# uncomment to save plot as pdf
#plt.savefig('EmpiricalPCalgorithmFC.pdf', bbox_inches='tight')

mat = m_PC
mat = np.mean(mat,axis=2)
m = abs(mat) > 0.005
mat = np.multiply(mat,m)
fig = cfc.plotConnectivityMatrix(mat,
                           methodTitle='PC algorithm FC',
                           functionalNetworks = True,
                           networkColorBar_x = True,
                           networkColorBar_y = True,
                           networkLabels = True,
                           )
# uncomment to save plot as pdf
#plt.savefig('EmpiricalPCadjacenciesFC.pdf', bbox_inches='tight')

### plot task-evoked activations: regions x task conditions

In [ ]:
act_subjectMedian = np.median(activation_data,axis=2)
plt_act.plot_activations(act_subjectMedian,'Actual activations',
                        functional_networks=True, no_ylabel=False, network_color_bar=True)
# uncomment to save plot as pdf
#plt.savefig('EmpiricalActualActivations.pdf', bbox_inches='tight')


act_subjectMedian = np.median(pred_act_corr,axis=2)
plt_act.plot_activations(act_subjectMedian,'Correlation FC\n predicted activations',
                        functional_networks=True, no_ylabel=False, network_color_bar=True)
# uncomment to save plot as pdf
#plt.savefig('EmpiricalCorrelationFCPredictedActivations.pdf', bbox_inches='tight')

act_subjectMedian = np.median(pred_act_PC,axis=2)
plt_act.plot_activations(act_subjectMedian,'PC algorithm FC\n predicted activations',
                        functional_networks=True, no_ylabel=False, network_color_bar=True)
# uncomment to save plot as pdf
#plt.savefig('EmpiricalCorrelationFCPredictedActivations.pdf', bbox_inches='tight')